In [ ]:
class Node(object):
    def __init__(object, parent, n_children=0):
        self.parent = parent
        self.n_children = n_children
        self.children = []
        for _ in range(self.n_children):
            self.children.append(Node(self))

In [11]:
import numpy as np
class DecisionTree(object):
    def __init__(self, eta=0.01, n_iter=50, random_state=1):
        self.eta = eta
        self.n_iter = n_iter
        self.random_state = random_state
    def gini(self, t):  # gini impuruty, t is the set of labels
        tags = np.unique(t)
        ig = 0
        for i in tags:
            ig += np.where(tags != i, 1, 0).sum()
        ig /= (len(tags)**(2))
        return ig
    def entropy(self, t):
        
        

In [12]:
import numpy as np
import pandas as pd  
df_trainX = pd.read_csv("/home/jiarui/Documents/3314/COMP3314_a1/dataset_files/car_X_train.csv", header=0, error_bad_lines=False)
df_trainy = pd.read_csv("/home/jiarui/Documents/3314/COMP3314_a1/dataset_files/car_y_train.csv", header=0, error_bad_lines=False)
df_testX = pd.read_csv("/home/jiarui/Documents/3314/COMP3314_a1/dataset_files/car_X_test.csv", header=0, error_bad_lines=False)
df_testy = pd.read_csv("/home/jiarui/Documents/3314/COMP3314_a1/dataset_files/car_y_test.csv", header=0, error_bad_lines=False)
X_train = df_trainX.iloc[:].values
y_train = df_trainy.iloc[:, 0].values 
X_test = df_testX.iloc[:].values
y_test = df_testy.iloc[:, 0].values

markers = ('o', 'x', 's', '^', 'v')
colors = ('red', 'blue', 'lightgreen', 'gray', 'cyan')
labels = np.unique(y_train)
df_trainX.head(10)

,buying,maint,doors,persons,lug_boot,safety
0,med,med,5more,4,big,high
1,high,high,3,more,small,low
2,low,med,3,4,med,med
3,med,med,5more,2,big,high
4,med,high,4,2,big,med
5,med,vhigh,2,more,big,low
6,med,high,3,2,small,high
7,vhigh,low,2,2,med,low
8,med,vhigh,2,4,big,low
9,med,low,3,2,small,high


In [23]:
dt = DecisionTree()
dt.gini(y_test)

0.75